В этом ноутбуке обучим модель  BERT with Talking-Heads Attention and Gated GELU (talkheads_ggelu_bert) распознавать спам в email

In [1]:
model_name = 'talkheads_ggelu_bert'
train_dataset_name = 'spam emails'

In [2]:
!pip install tensorflow-text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.5 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully u

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Загружаем данные

In [4]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/data_for_colab/spam_emails.csv', encoding = "ISO-8859-1")
df

Mounted at /content/drive


,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [5]:
df.drop(columns=['Unnamed: 0', 'label'], inplace=True)
df.columns = ['DATA_COLUMN', 'IS_SPAM']

In [6]:
df

,DATA_COLUMN,IS_SPAM
0,Subject: enron methanol ; meter # : 988291\r\n...,0
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...
5166,Subject: put the 10 on the ft\r\nthe transport...,0
5167,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,Subject: industrial worksheets for august 2000...,0


In [7]:
df['DATA_COLUMN'] = df['DATA_COLUMN'].apply(lambda x: x.replace('\r\n', ' ').replace('\n', ' '))

In [8]:
df

,DATA_COLUMN,IS_SPAM
0,Subject: enron methanol ; meter # : 988291 thi...,0
1,"Subject: hpl nom for january 9 , 2001 ( see at...",0
2,"Subject: neon retreat ho ho ho , we ' re aroun...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs this deal is to b...,0
...,...,...
5166,Subject: put the 10 on the ft the transport vo...,0
5167,Subject: 3 / 4 / 2000 and following noms hpl c...,0
5168,Subject: calpine daily gas nomination > > juli...,0
5169,Subject: industrial worksheets for august 2000...,0


In [9]:
df['IS_SPAM'].value_counts()

0    3672
1    1499
Name: IS_SPAM, dtype: int64

In [10]:
df_positive = df[df['IS_SPAM']==1]

In [11]:
df_negative = df[df['IS_SPAM']==0]

Создаем тестовую и обучающую выборки

In [12]:
# Тестовая выборка
n_test = df_negative.shape[0] // 2
df_negative_test = df_negative.tail(n_test)
n_test = df_positive.shape[0] // 2
df_positive_test = df_positive.tail(n_test)

In [13]:
df_negative_test.shape

(1836, 2)

In [14]:
df_positive_test.shape

(749, 2)

In [15]:
df_positive_test

,DATA_COLUMN,IS_SPAM
2699,Subject: your mother knows you watch girls moa...,1
2702,Subject: 65 % off for all new software . pyrrh...,1
2703,Subject: [ wrenches ] 68 % off dreamweaver mx ...,1
2706,Subject: he reached around and fingered me whi...,1
2709,"Subject: best meds around , paliourg haven ' t...",1
...,...,...
5159,Subject: pictures streamlined denizen ajar cha...,1
5161,Subject: penny stocks are about timing nomad i...,1
5162,Subject: anomaly boys from 3881 uosda apaprove...,1
5164,Subject: slutty milf wants to meet you take th...,1


In [16]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [17]:
df_balanced_test.sample(10)

,DATA_COLUMN,IS_SPAM
3196,Subject: revised - april tenaska sheet megan /...,0
5087,Subject: fw : please read . . don ' t delete -...,0
2785,Subject: unify close schedule the following is...,0
2678,Subject: unify passwords will be reset under s...,0
3722,"Subject: enron / hpl actuals for june 6 , 2000...",0
3564,Subject: a better way is at handheldmed . com ...,1
3619,Subject: replica rolex and other popular brand...,1
4486,Subject: here ' s my performance evaluation ch...,0
4298,"Subject: enron actuals for june 22 , 2000 teco...",0
4614,Subject: ciallis softabs onlly $ 3 . 99 % q ne...,1


In [18]:
df_balanced_test['IS_SPAM'].value_counts()

0    1836
1     749
Name: IS_SPAM, dtype: int64

In [19]:
# Обучающая выборка
n_train = df_negative.shape[0] // 2
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 2
df_positive_train = df_positive.head(n_train)

In [20]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [21]:
df_balanced_train['IS_SPAM'].value_counts()

0    1836
1     749
Name: IS_SPAM, dtype: int64

In [22]:
df_balanced_train.sample(10)

,DATA_COLUMN,IS_SPAM
1197,Subject: ena sales on hpl just to update you o...,0
2423,"Subject: david triplett , * 75 % off for all n...",1
1276,Subject: 18 yearold cocksuckers ! download hot...,1
622,Subject: ces - carthage i decreased volumes at...,0
136,"Subject: re : tuesday , december 26 th i will ...",0
327,"Subject: viicodin , via - gra are che . ap her...",1
1860,"Subject: pw 9 o 3 inx sat , 19 jun 2004 01 : 1...",1
1916,Subject: swjlmiqqt the fountain of youth ajuin...,1
1957,Subject: physical curve mappings _ sitara hey ...,0
1031,Subject: ( no subject ) hey daren its me . hav...,0


In [23]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['IS_SPAM'].squeeze()

In [24]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['IS_SPAM'].squeeze()

Работаем с моделью

In [25]:
talkheads_ggelu_bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3') # uses same thing as bert

In [26]:
talkheads_ggelu_bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/2", trainable=False)

In [27]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = talkheads_ggelu_bert_preprocess(text_input)
outputs = talkheads_ggelu_bert_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [28]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [29]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [31]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [32]:
history = model.fit(X_train, y_train, epochs=15)

Epoch 1/15
81/81 [==============================] - 46s 432ms/step - loss: 0.5699 - accuracy: 0.7103 - precision: 0.5000 - recall: 0.0280
Epoch 2/15
81/81 [==============================] - 36s 440ms/step - loss: 0.4757 - accuracy: 0.7567 - precision: 0.9110 - recall: 0.1776
Epoch 3/15
81/81 [==============================] - 35s 429ms/step - loss: 0.4206 - accuracy: 0.8031 - precision: 0.9545 - recall: 0.3364
Epoch 4/15
81/81 [==============================] - 35s 435ms/step - loss: 0.3834 - accuracy: 0.8364 - precision: 0.9335 - recall: 0.4686
Epoch 5/15
81/81 [==============================] - 35s 432ms/step - loss: 0.3580 - accuracy: 0.8534 - precision: 0.9384 - recall: 0.5287
Epoch 6/15
81/81 [==============================] - 36s 447ms/step - loss: 0.3321 - accuracy: 0.8696 - precision: 0.9364 - recall: 0.5901
Epoch 7/15
81/81 [==============================] - 35s 432ms/step - loss: 0.3214 - accuracy: 0.8712 - precision: 0.9160 - recall: 0.6115
Epoch 8/15
81/81 [================

In [33]:
y_predicted = model.predict(X_test)
y_predicted


81/81 [==============================] - 35s 427ms/step


array([[0.35684294],
       [0.1218214 ],
       [0.01256138],
       ...,
       [0.71438646],
       [0.98040867],
       [0.5069603 ]], dtype=float32)

In [34]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [35]:
y_test

2536    0
2537    0
2539    0
2540    0
2542    0
       ..
5159    1
5161    1
5162    1
5164    1
5170    1
Name: IS_SPAM, Length: 2585, dtype: int64

In [36]:
accuracy_score(y_test, y_predicted)

0.9013539651837524

In [37]:
precision_score(y_test, y_predicted)


0.9288194444444444

In [38]:
recall_score(y_test, y_predicted)

0.7142857142857143

In [39]:
f1_score(y_test, y_predicted)

0.8075471698113208

In [40]:
df_results_on_test = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [41]:
index = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'
df_results_on_test.loc[index, 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_on_test.loc[index, 'precision'] = precision_score(y_test, y_predicted)
df_results_on_test.loc[index, 'recall'] = recall_score(y_test, y_predicted)
df_results_on_test.loc[index, 'f1_score'] =  f1_score(y_test, y_predicted)

In [42]:
df_results_on_test

,accuracy,precision,recall,f1_score
talkheads_ggelu_bert trained on spam emails and tested on spam emails dataset,0.901354,0.928819,0.714286,0.807547


Сохраним обученную модель

In [43]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/talkheads_ggelu_bert_trained_on_spam_emails_20_january'

In [44]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/talkheads_ggelu_bert_trained_on_spam_emails_20_january


In [45]:
model.save(saved_model_path, include_optimizer=True) 

Сохраним датафреймы с результатами на обучающей выборке и на тестовой выборке

Сначала результаты для обучающей выборки

In [46]:
name_for_train_csv = model_name + ' trained on ' + train_dataset_name + ' quality on train dataset'

In [47]:
def calculate_f1_score(precision, recall):
    F1 = 2 * (precision * recall) / (precision + recall)
    return F1

In [48]:
df_with_train_quality = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [49]:
for el in ['accuracy', 'precision', 'recall']:
    df_with_train_quality.loc[name_for_train_csv, el] = history.history[el][-1]
df_with_train_quality.loc[name_for_train_csv, 'f1_score'] = calculate_f1_score(history.history['precision'][-1], history.history['recall'][-1])

In [50]:
df_with_train_quality

,accuracy,precision,recall,f1_score
talkheads_ggelu_bert trained on spam emails quality on train dataset,0.907157,0.927731,0.736983,0.821429


In [51]:
df_with_train_quality.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/train_quality/' + name_for_train_csv + '.csv')

Теперь результаты для тестовой выборки

In [52]:
name_for_test_csv = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'

In [53]:
df_results_on_test.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/test_quality/' + name_for_test_csv + '.csv')